In [7]:
#!/usr/bin/env python
# coding: utf-8

import pyautocad
from pyautocad import Autocad, APoint,aDouble, ACAD
import logging,os,time

def cad_selector(cad_doc,set_name,promp="Select objects"):
    """Phương thức chọn đối tượng trong file CAD"""
    print(promp)
    cad_doc.Utility.Prompt(u"%s\n" % promp)
    try:
        cad_doc.SelectionSets.Item(set_name).Delete()
    except Exception as ex:
        pass
    selection = cad_doc.SelectionSets.Add(set_name)
    selection.SelectOnScreen()
    return selection

def get_items(cad_doc,in_text = 'AcDb',in_xref = False,in_block = False):
    ms = cad_doc.ModelSpace
    list_items = [ms.Item(i) for i in range(ms.Count) if in_text in ms.Item(i).ObjectName]
    return list_items


    
def open_file(acad,path = None):    
    if path == None:
        while True:
            path = input('Đường dẫn file CAD: ')
            if os.path.isfile(path):
                print('Đã load Path')
                break
            else:
                print('Path không tồn tại')    
    file_name = path.split("\\")[-1]
    # print(f"File name: {file_name}")
    docs = acad.app.Documents # print(cad_doc.Name)
    doc_names = [d.Name for d in docs]
    while True:
        if not file_name in doc_names:    
            acad.app.Documents.Open(path)
            docs = acad.app.Documents # print(cad_doc.Name)
            doc_names = [d.Name for d in docs]    
        for d in docs:
            if d.Name == file_name:
                cad_doc = d
                print('Thành công mở file xref')
                break
        if cad_doc != None:
            break
        else:
            print('Không thể mở file xref')
    acad.app.ZoomAll()
    cad_doc.Regen (True)
    return cad_doc

def show_hide_items_in_doc(doc,item_type = "Text",show =False):
    ms = doc.ModelSpace
    block_names = []
    for i in range(ms.Count):
        try: # xử lí các item trong Doc
            item = ms.Item(i)
            if item_type in item.ObjectName:
                item.Visible = show # ẨN Item
            if 'AcDbBlockReference' in item.ObjectName: # lấy ds tên block referent
                block_names.append(item.Name)
        except Exception as ex:
            pass
    # xử lí các BLOCK
    blocks = doc.Blocks
    for block in blocks:
        try: 
            if block.Name in block_names:
                show_hide_items_in_block(block,item_type = item_type, show = show)
        except Exception as ex:
            # print(ex)
            pass 
    doc.Regen (True)

def show_hide_items_in_block(block,item_type = "Text",show =False): 
    for i in range(block.Count):
        try: # xử lí các item trong block
            item = block.Item(i)
            if item_type in item.ObjectName:
                item.Visible = show # ẨN item
            if 'AcDbBlockReference' in item.ObjectName: # xử lí tiếp các Block con
                show_hide_items_in_block(item,item_type = item_type, show = show)
        except Exception as ex:
            # print (ex)
            pass

def set_color_items_in_doc(doc,item_type = "Text",R = 192,G = 192 , B = 192):
    ms = doc.ModelSpace
    block_names = []
    color = ms.Item(0).TrueColor
    color.SetRGB(R,G,B)
    for i in range(ms.Count):
        try: # xử lí các item trong Doc
            item = ms.Item(i)
            if item_type in item.ObjectName:
                item.TrueColor = color
            if 'AcDbBlockReference' in item.ObjectName: # lấy ds tên block referent
                block_names.append(item.Name)
        except Exception as ex:
            pass
    # xử lí các BLOCK
    blocks = doc.Blocks
    for block in blocks:
        try: 
            if block.Name in block_names:
                set_color_items_in_block(block,item_type = item_type, R = R,G = G , B = B)
        except Exception as ex:
            # print(ex)
            pass 
    doc.Regen (True)
def set_color_items_in_block(block,item_type = "Text",R = 192,G = 192 , B = 192): 
    color = block.Item(0).TrueColor
    color.SetRGB(R,G,B)
    for i in range(block.Count):
        try: # xử lí các item trong block
            item = block.Item(i)
            if item_type in item.ObjectName:
                item.TrueColor = color
            if 'AcDbBlockReference' in item.ObjectName: # xử lí tiếp các Block con
                set_color_items_in_block(item,item_type = item_type,  R = R,G = G , B = B)

        except Exception as ex:
            # print (ex)
            pass
    


In [33]:
#-------------------------------------------------------------------#
if __name__ ==  '__main__':
    """"""
    # DEBUGGER
    debug_file_name = f"{os.getcwd()}\\debug\\debug.txt" #<---  SỬ DUNG CHO 1 LẦN DEBUG
    # debug_file_name = f"{os.getcwd()}\\debug\\debug-{time.strftime('%y%m%d %H%M%S',time.localtime(time.time()))}.txt" #<---  SỬ DUNG CHO NHIỀU LẦN DEBUG
    logging.basicConfig(filename = debug_file_name,level=logging.INFO, format='%(message)s')
    # logging.disable(logging.CRITICAL) ########### <---  UNCOMMNEND khi không cần debug nữa
    logging.info('Program START')
    
    # MỞ AUTOCAD
    acad = Autocad(create_if_not_exists=True, visible=True)
    if acad:  logging.info('Autocad Opened')
    
    # MỞ DOCUMENTS AUTOCAD
    cad_doc_list = list(acad.app.Documents)
    
    # SHOW LIST DOC
    [print(f"{i} : {cad_doc_list[i].Name}") for i in range(len(cad_doc_list))]
    
    # HỎI CHỌN DOCUMENT
    while True:
        try:
            ask_doc = input("Vui lòng chọn Document: ")    
            if ask_doc == '0' or int(ask_doc):
                cad_doc = cad_doc_list[int(ask_doc)]
                break
        except:
            continue
    
    
    logging.info(cad_doc.Name)

    # MODEL SPACE
    model_space = cad_doc.ModelSpace
    paper_space = cad_doc.PaperSpace
    logging.info([model_space.Name,paper_space.name])
    """
    # LAYOUT
    layouts = list(cad_doc.Layouts)
    [print(f"{i} : {layouts[i].Name}") for i in range(len(layouts))]    
    logging.info([layout.Name for layout in layouts])
    # HỎI CHỌN LAYOUT
    while True:
        try:
            ask_layout = input("Vui lòng chọn Layout: ")    
            if ask_layout == '0' or int(ask_layout):
                layout = layouts[int(ask_layout)]
                break
        except:
            continue
    logging.info(layout.Name)
    
    """
    
    """
    print('\n'.join(["0 : Show","1 : Hide"]))
    while True:
        ask_show_hide = input('Bạn muốn show or hide ? ')
        if ask_show_hide == "0":
            show = True
            break
        elif  ask_show_hide == "1":
            show = False
            break

    show_hide_items_in_doc(cad_doc,item_type = "Dimension", show = show)"""

0 : Drawing2.dwg
1 : x-01-xuong 2.dwg


Vui lòng chọn Document:  0


In [ ]:
   # VIEWPORT BY ACTIVE FIRST
    print("Bạn cần active vào Viewport bạn muốn xử lí")
    active_viewport = cad_doc.ActivePViewport
    logging.info(f"{active_viewport.ObjectName} {round(active_viewport.Width)} x {round(active_viewport.Height)}")

In [24]:
#-------------------------------------------------------------------------------------#
# TRYING TO OPEN XREF, BUT FAIL WITH BIG CAD FILE
all_items = [model_space.Item(i) for i in range(model_space.Count)]
test_xref = None
name = input("Nhập tên file xref: ")
for i in all_items:
    try:
        if 'Block' in i.ObjectName and name in i.Path:
            test_xref = i
    except:
        pass
full_name = cad_doc.FullName
dir_folder = full_name[:-len(full_name.split('\\')[-1])]
xref_path = test_xref.Path

if xref_path[0] == '.':
    xref_path = f"{dir_folder}{xref_path[2:]}"
time.sleep(1)
print(xref_path)


Nhập tên file xref:  220210602A01XUONG2FACTORY2SHOP


K:\_WFH DANON\_DOC\2.6-Combine\2.6-Combine\2021-06-03 Combine dot dieu chinh sau chot mb kien truc xuogn 2\220210602A01XUONG2FACTORY2SHOP.dwg


In [25]:
xref_doc = open_file(acad,path = xref_path)
time.sleep(15)
hide_items_in_doc(xref_doc,item_type = "Text")

COMError: (-2147352567, 'Exception occurred.', ('Error Decrypting Data', 'AutoCAD', 'C:\\Program Files\\Autodesk\\AutoCAD 2019\\HELP\\OLE_ERR.CHM', -2145386153, None))

In [23]:
all_items = [model_space.Item(i) for i in range(model_space.Count)]
for i in all_items:
    try:
        print(i.Name)
    except:
        pass

x-01-xuong 2
220210602A01XUONG2FACTORY2SHOP
5A01XUONG 2FACTORY 2CEILING
block 1
Drawing2
Drawing3


In [25]:
dim = cad_selector(cad_doc,'dim',promp="Select objects")
dim.Name

Select objects


'dim'

In [32]:
color = model_space.Item(0).TrueColor
color.SetRGB(255,0,0)
dim.Item(0).TrueColor = color
dim.Item(0).DimensionLineColor = color.ColorIndex
dim.Item(0).ExtensionLineColor = color.ColorIndex
dim.Item(0).TextColor = color.ColorIndex

In [35]:
for i in range(model_space.Count):
    try:
        item = model_space.Item(i)
        print(item.Name)
    except:
        pass

block 1
BLOCK BLOCK BLOCK


In [36]:
block = cad_selector(cad_doc,'block_',promp="Select objects")

Select objects


'dim'

In [40]:
block[0].Name

'BLOCK BLOCK BLOCK'

In [45]:
# GLOBAL VAR
list_dic_block = []
def create_list_dic_block(doc):
    global list_dic_block
    blocks = doc.Blocks
    dic = {}
    for i in range(blocks.Count):
        block = blocks[i]
        try: 
            dic[i] = [block,False]   
            list_dic_block.append(dic)
        except Exception as ex:
            pass 
    return list_dic_block

create_list_dic_block(cad_doc)
list_dic_block

for d in list_dic_block:
    print(d)

{0: [<comtypes.client.lazybind.Dispatch object at 0x000002003CD80D90>, False], 1: [<comtypes.client.lazybind.Dispatch object at 0x000002007D563400>, False], 2: [<comtypes.client.lazybind.Dispatch object at 0x000002007D563100>, False], 3: [<comtypes.client.lazybind.Dispatch object at 0x000002007D3A3C70>, False], 4: [<comtypes.client.lazybind.Dispatch object at 0x000002007D3A36D0>, False], 5: [<comtypes.client.lazybind.Dispatch object at 0x000002007D3959A0>, False], 6: [<comtypes.client.lazybind.Dispatch object at 0x000002007D395070>, False], 7: [<comtypes.client.lazybind.Dispatch object at 0x000002007D395580>, False], 8: [<comtypes.client.lazybind.Dispatch object at 0x000002007D395EB0>, False], 9: [<comtypes.client.lazybind.Dispatch object at 0x000002007D395C40>, False], 10: [<comtypes.client.lazybind.Dispatch object at 0x000002007D3956D0>, False], 11: [<comtypes.client.lazybind.Dispatch object at 0x000002007D395340>, False], 12: [<comtypes.client.lazybind.Dispatch object at 0x000002007